# Iframe Isolation Security Test

This notebook verifies that HTML outputs are properly isolated from Tauri APIs.
All tests should show ✅ if isolation is working correctly.

## Security Verification

These cells test that the isolated iframe cannot access Tauri APIs or the parent window.

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<div id="tauri-test" style="padding: 12px; border-radius: 4px; font-family: system-ui;">
  <strong>Test 1: Tauri API Access</strong>
  <div id="tauri-result" style="margin-top: 8px;">Checking...</div>
  <script>
    (function() {
      const result = document.getElementById('tauri-result');
      const hasTauri = typeof window.__TAURI__ !== 'undefined';
      const hasInternals = typeof window.__TAURI_INTERNALS__ !== 'undefined';
      
      if (hasTauri || hasInternals) {
        result.innerHTML = '❌ <span style="color: #ef4444;">FAILED - Tauri APIs are accessible!</span>';
        result.innerHTML += '<br><small>window.__TAURI__: ' + hasTauri + '</small>';
        result.innerHTML += '<br><small>window.__TAURI_INTERNALS__: ' + hasInternals + '</small>';
      } else {
        result.innerHTML = '✅ <span style="color: #22c55e;">PASSED - Tauri APIs are blocked</span>';
      }
    })();
  </script>
</div>
'''))

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<div id="parent-test" style="padding: 12px; border-radius: 4px; font-family: system-ui;">
  <strong>Test 2: Parent Document Access</strong>
  <div id="parent-result" style="margin-top: 8px;">Checking...</div>
  <script>
    (function() {
      const result = document.getElementById('parent-result');
      let canAccessParent = false;
      let errorMsg = '';
      
      try {
        // This should throw a cross-origin error if properly isolated
        const test = window.parent.document.body;
        canAccessParent = true;
      } catch (e) {
        errorMsg = e.message;
      }
      
      if (canAccessParent) {
        result.innerHTML = '❌ <span style="color: #ef4444;">FAILED - Can access parent document!</span>';
      } else {
        result.innerHTML = '✅ <span style="color: #22c55e;">PASSED - Parent document access blocked</span>';
        result.innerHTML += '<br><small style="color: #888;">Error: ' + errorMsg + '</small>';
      }
    })();
  </script>
</div>
'''))

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<div id="storage-test" style="padding: 12px; border-radius: 4px; font-family: system-ui;">
  <strong>Test 3: Parent Storage Access</strong>
  <div id="storage-result" style="margin-top: 8px;">Checking...</div>
  <script>
    (function() {
      const result = document.getElementById('storage-result');
      let canAccessStorage = false;
      let errorMsg = '';
      
      try {
        // This should throw a cross-origin error if properly isolated
        const test = window.parent.localStorage.getItem('test');
        canAccessStorage = true;
      } catch (e) {
        errorMsg = e.message;
      }
      
      if (canAccessStorage) {
        result.innerHTML = '❌ <span style="color: #ef4444;">FAILED - Can access parent storage!</span>';
      } else {
        result.innerHTML = '✅ <span style="color: #22c55e;">PASSED - Parent storage access blocked</span>';
        result.innerHTML += '<br><small style="color: #888;">Error: ' + errorMsg + '</small>';
      }
    })();
  </script>
</div>
'''))

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<div id="origin-test" style="padding: 12px; border-radius: 4px; font-family: system-ui;">
  <strong>Test 4: Window Origin</strong>
  <div id="origin-result" style="margin-top: 8px;">Checking...</div>
  <script>
    (function() {
      const result = document.getElementById('origin-result');
      const origin = window.origin || window.location.origin;
      
      // In a properly isolated blob URL iframe, origin should be "null" (opaque)
      const isOpaque = origin === 'null' || origin === null;
      
      if (isOpaque) {
        result.innerHTML = '✅ <span style="color: #22c55e;">PASSED - Opaque origin (isolated)</span>';
        result.innerHTML += '<br><small style="color: #888;">Origin: ' + origin + '</small>';
      } else {
        result.innerHTML = '⚠️ <span style="color: #eab308;">WARNING - Has specific origin</span>';
        result.innerHTML += '<br><small style="color: #888;">Origin: ' + origin + '</small>';
        result.innerHTML += '<br><small>This may indicate iframe is not fully isolated</small>';
      }
    })();
  </script>
</div>
'''))

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<div id="summary-test" style="padding: 16px; border-radius: 8px; font-family: system-ui; margin-top: 16px;">
  <strong style="font-size: 1.1em;">Overall Security Status</strong>
  <div id="summary-result" style="margin-top: 12px; font-size: 1.2em;">Evaluating...</div>
  <script>
    (function() {
      const result = document.getElementById('summary-result');
      
      // Run all checks
      const hasTauri = typeof window.__TAURI__ !== 'undefined';
      const hasInternals = typeof window.__TAURI_INTERNALS__ !== 'undefined';
      
      let canAccessParent = false;
      try {
        const test = window.parent.document.body;
        canAccessParent = true;
      } catch (e) {}
      
      let canAccessStorage = false;
      try {
        const test = window.parent.localStorage.getItem('test');
        canAccessStorage = true;
      } catch (e) {}
      
      const isIsolated = !hasTauri && !hasInternals && !canAccessParent && !canAccessStorage;
      
      if (isIsolated) {
        result.innerHTML = '🔒 <span style="color: #22c55e; font-weight: bold;">FULLY ISOLATED</span>';
        result.innerHTML += '<p style="font-size: 0.9em; color: #888;">Notebook outputs are running in a secure sandbox.</p>';
      } else {
        result.innerHTML = '⚠️ <span style="color: #ef4444; font-weight: bold;">ISOLATION FAILED</span>';
        result.innerHTML += '<p style="font-size: 0.9em; color: #888;">Some security checks did not pass. Check individual tests above.</p>';
      }
    })();
  </script>
</div>
'''))

## Output Rendering Tests

These cells test various output types to verify they render correctly in the isolated frame.

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<div style="padding: 12px; border: 1px solid #444; border-radius: 4px;">
  <h3 style="margin-top: 0;">Basic HTML Rendering</h3>
  <p>This paragraph should render correctly.</p>
  <ul>
    <li>List item 1</li>
    <li>List item 2</li>
    <li>List item 3</li>
  </ul>
  <p><strong>Bold</strong>, <em>italic</em>, and <code>inline code</code>.</p>
</div>
'''))

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<table style="border-collapse: collapse; margin: 8px 0;">
  <thead>
    <tr>
      <th style="border: 1px solid #444; padding: 8px; background: #2a2a2a;">Name</th>
      <th style="border: 1px solid #444; padding: 8px; background: #2a2a2a;">Value</th>
      <th style="border: 1px solid #444; padding: 8px; background: #2a2a2a;">Status</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid #444; padding: 8px;">Alpha</td>
      <td style="border: 1px solid #444; padding: 8px;">42</td>
      <td style="border: 1px solid #444; padding: 8px; color: #22c55e;">Active</td>
    </tr>
    <tr>
      <td style="border: 1px solid #444; padding: 8px;">Beta</td>
      <td style="border: 1px solid #444; padding: 8px;">17</td>
      <td style="border: 1px solid #444; padding: 8px; color: #eab308;">Pending</td>
    </tr>
    <tr>
      <td style="border: 1px solid #444; padding: 8px;">Gamma</td>
      <td style="border: 1px solid #444; padding: 8px;">99</td>
      <td style="border: 1px solid #444; padding: 8px; color: #ef4444;">Error</td>
    </tr>
  </tbody>
</table>
'''))

In [ ]:
from IPython.display import HTML, display

display(HTML('''
<div id="script-test" style="padding: 12px; border-radius: 4px;">
  <strong>Script Execution Test</strong>
  <div id="script-result" style="margin-top: 8px;">Waiting for script...</div>
  <script>
    (function() {
      const result = document.getElementById('script-result');
      result.innerHTML = '✅ <span style="color: #22c55e;">Script executed successfully!</span>';
      result.innerHTML += '<br><small>Timestamp: ' + new Date().toISOString() + '</small>';
    })();
  </script>
</div>
'''))

## Markdown Test

In [ ]:
from IPython.display import Markdown, display

display(Markdown('''
# Markdown Heading

This is a **markdown** cell with:

- Bullet points
- More bullet points
- Even more bullet points

And some `inline code` along with:

```python
def hello():
    return "world"
```

> A blockquote for emphasis
'''))

## SVG Test

In [ ]:
from IPython.display import SVG, display

svg_content = '''
<svg width="200" height="100" xmlns="http://www.w3.org/2000/svg">
  <rect x="10" y="10" width="80" height="80" fill="#3b82f6" rx="8"/>
  <rect x="110" y="10" width="80" height="80" fill="#22c55e" rx="8"/>
  <text x="50" y="55" text-anchor="middle" fill="white" font-family="system-ui">A</text>
  <text x="150" y="55" text-anchor="middle" fill="white" font-family="system-ui">B</text>
</svg>
'''

display(SVG(svg_content))

## JSON Test

In [ ]:
from IPython.display import JSON, display

data = {
    "name": "test-isolation",
    "version": "1.0.0",
    "features": ["security", "rendering", "scripting"],
    "config": {
        "isolated": True,
        "sandbox": "allow-scripts"
    }
}

display(JSON(data))

## Mixed Output Test

This cell produces multiple outputs that should all render together in the isolated frame.

In [ ]:
from IPython.display import HTML, display

# First output: plain text
print("This is stdout output")

# Second output: HTML
display(HTML('<div style="padding: 8px; background: #1e3a1e; border-radius: 4px; color: white;">HTML output in the middle</div>'))

# Third output: more stdout
print("More stdout after HTML")

# Fourth output: another HTML
display(HTML('<div style="padding: 8px; background: #1e2a3a; border-radius: 4px; color: white;">Another HTML block</div>'))

print("Final stdout line")


## ANSI Colors Test

In [ ]:
# ANSI color codes
colors = [
    ('31', 'Red'),
    ('32', 'Green'),
    ('33', 'Yellow'),
    ('34', 'Blue'),
    ('35', 'Magenta'),
    ('36', 'Cyan'),
]

print('ANSI Colors:')
for code, name in colors:
    print(f'\x1b[{code}m{name}\x1b[0m', end='  ')
print()

print('\n\x1b[1mBold text\x1b[0m')
print('\x1b[1;32mBold green\x1b[0m')
print('\x1b[1;31mBold red\x1b[0m')

## Image Test

In [ ]:
from IPython.display import Image, display
import base64

# Create a simple 50x50 PNG image (solid blue)
# This is a minimal valid PNG
png_data = base64.b64decode(
    'iVBORw0KGgoAAAANSUhEUgAAADIAAAAyCAIAAACRXR/mAAAAGklEQVR4nO3BMQEAAADCoPVP7WsI'
    'oAAALg4ADwABWIsYkQAAAABJRU5ErkJggg=='
)

display(Image(data=png_data, format='png', width=50, height=50))
print("Image should appear above (small square)")